#### Etape 2.1 : Nettoyage des donnees meteo
- Charger `meteo_raw.csv` avec Pandas
- Standardiser les formats de dates
- Convertir les colonnes numeriques en gerant les erreurs
- Corriger les valeurs aberrantes :
  - Temperatures hors [-40, 50] -> NaN puis interpolation
  - Humidite hors [0, 100] -> clipping
  - Rayonnement solaire negatif -> 0
- Traiter les valeurs manquantes :
  - Interpolation lineaire pour temperature et humidite
  - Forward fill pour precipitation
- Ajouter des colonnes temporelles (jour, mois, saison, jour de semaine)

**Livrables** :
- Notebook `04_nettoyage_meteo_pandas.ipynb`
- Dataset nettoye `output/meteo_clean.csv`
- Rapport avant/apres nettoyage (completude par colonne)

In [1]:
import pandas as pd

df = pd.read_csv("../data_ecf/meteo_raw.csv", sep=",")

print(f"Dataset brut : {df}")

# Standardiser les formats de dates
formats = [
    "%Y-%m-%d %H:%M:%S",  
    "%d/%m/%Y %H:%M",     
    "%Y/%m/%d %H:%M:%S",  
    "%d-%m-%Y %H:%M:%S", 
    "%m/%d/%Y %H:%M:%S",   
    "%Y-%m-%dT%H:%M:%S"
]

def parse_dates(ts):
    for fmt in formats:
        try:
            return pd.to_datetime(ts, format=fmt)
        except:
            continue
    return pd.NaT

df['timestamp'] = df['timestamp'].apply(parse_dates)

print(df)

print(df)

# Convertir les colonnes numériques en gérant les erreurs
columns = ['temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2','vitesse_vent_kmh', 'precipitation_mm']

for col in columns:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

# Corriger les valeurs aberrantes

# Température hors [-40, 50] -> NaN puis interpolation
df.loc[~df['temperature_c'].between(-40, 50), 'temperature_c'] = pd.NA
df['temperature_c'] = df['temperature_c'].interpolate(method='linear').round(2)

# Humidité hors [0, 100] -> clipping puis interpolation
df['humidite_pct'] = df['humidite_pct'].clip(0, 100)
df['humidite_pct'] = df['humidite_pct'].interpolate(method='linear')

# Rayonnement solaire négatif -> 0
df['rayonnement_solaire_wm2'] = df['rayonnement_solaire_wm2'].clip(lower=0)

# Précipitations -> forward fill
df['precipitation_mm'] = df['precipitation_mm'].ffill()

#  Ajouter des colonnes temporelles (jour, mois, saison, jour de semaine)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['jour'] = df['timestamp'].dt.day
df['mois'] = df['timestamp'].dt.month_name()
df['annee'] = df['timestamp'].dt.year

print(f"Dataset nettoyé : {df}")

# Sauvegarder le dataset nettoyé
df.to_csv("../output/meteo_clean.csv", index=False)

Dataset brut :               commune            timestamp temperature_c  humidite_pct  \
0       Saint-Etienne  09/15/2024 15:00:00          17.1         143.3   
1            Bordeaux     21/07/2023 15:00          19.6          50.6   
2         Montpellier  2023-09-18 20:00:00          18.3          65.7   
3            Le Havre  01/03/2024 22:00:00           3.7          94.9   
4               Lille     29/10/2024 20:00          14.0          42.9   
...               ...                  ...           ...           ...   
252607       Toulouse     14/04/2024 19:00          14,1          81.3   
252608    Montpellier  03/13/2024 03:00:00          14.1          75.0   
252609       Bordeaux  06/04/2023 03:00:00          23.2          30.6   
252610  Saint-Etienne     29/11/2023 06:00          17.5          77.2   
252611           Lyon     18/03/2023 23:00           9.2          83.4   

        rayonnement_solaire_wm2  vitesse_vent_kmh  precipitation_mm  
0                         